In [81]:
# coding: utf-8
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.pipeline import Pipeline

from sklearn.naive_bayes import MultinomialNB
from nltk import word_tokenize
from nltk.tokenize import wordpunct_tokenize

import os
import random

In [82]:
def plot_sigmoid():
	X = np.linspace(-6, 6, 200)
	Y = 1 / (1 + np.exp(-X))

	plt.plot(X, Y)
	plt.grid(True)
	plt.xlabel('X')
	plt.ylabel('logistic function')
	plt.show()

In [83]:
negative = [f for f in os.listdir('./review_polarity 2/txt_sentoken/neg') if f.endswith('.txt')]

In [84]:
positive = [f for f in os.listdir('./review_polarity 2/txt_sentoken/pos') if f.endswith('.txt')]

In [85]:
arr_reviews = []

for review in negative:
    with open('./review_polarity 2/txt_sentoken/neg/' + review, 'r', encoding='utf-8') as r:
        dicti = {}
        text = r.read()
        dicti['class'] = '0'
        dicti['text'] = text
        arr_reviews.append(dicti)
        
for review in positive:
    with open('./review_polarity 2/txt_sentoken/pos/' + review, 'r', encoding='utf-8') as r:
        dicti = {}
        text = r.read()
        dicti['class'] = '1'
        dicti['text'] = text
        arr_reviews.append(dicti)

        
random.shuffle(arr_reviews)  # перемешиваем данные
# for line in arr:
#     if line['class'] == '1':
#         print(line)
    
df = pd.DataFrame(arr_reviews)  # создаем датасет


In [86]:
df.head()

,class,text
0,0,michael robbins' hardball is quite the cinemat...
1,0,you should have heard the old guys in the crow...
2,0,lucas was wise to start his star wars trilogy ...
3,0,"set in harlem during the great depression , ri..."
4,1,"david lynch's "" blue velvet "" begins and ends ..."


In [87]:
df.describe()

,class,text
count,2000,2000
unique,2,2000
top,1,"at first glance , i thought that the sword and..."
freq,1000,1


In [88]:
df.groupby('class').describe()

text
class                                                          
0     count                                                1000
      unique                                               1000
      top     at first glance , i thought that the sword and...
      freq                                                    1
1     count                                                1000
      unique                                               1000
      top      " it's not good to know too much about someon...
      freq                                                    1

In [89]:
#  Делим выборку в соотновении 80:20
rev_train, rev_test, class_train, class_test = train_test_split(df['text'], df['class'], test_size=0.2)

In [90]:
rev_train.shape, rev_test.shape, class_train.shape, class_test.shape

((1600,), (400,), (1600,), (400,))

In [91]:
# Векторизация
bow = CountVectorizer()
bow.fit_transform(rev_train)
bowed_rev_train = bow.transform(rev_train)  # тренировочные тексты
bowed_rev_test = bow.transform(rev_test)   # тестовые тексты

In [92]:
# Обучаем модель логистической регрессии
clf = LogisticRegression(class_weight='balanced')
clf.fit(bowed_rev_train, class_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [93]:
print(classification_report(class_test, clf.predict(bowed_rev_test)))

             precision    recall  f1-score   support

          0       0.84      0.87      0.86       185
          1       0.89      0.86      0.87       215

avg / total       0.87      0.86      0.87       400



In [94]:
clf.coef_

array([[  1.09569519e-02,  -1.28542592e-02,   8.72086557e-06, ...,
          1.07465888e-03,   1.70542903e-03,  -2.48964022e-03]])

In [95]:
clf.intercept_

array([-0.54294908])

In [96]:
clf.classes_

array(['0', '1'], dtype=object)

In [97]:
pd.crosstab(df['text'], df['class'], rownames=['text'])

class                                               0  1
text                                                    
 " . . . because i'm a scientist . \nthat's wha...  1  0
 " . . it's certainly more than just a monster ...  0  1
 " a breed apart " casts rutger hauer as a craz...  0  1
 " a bug's life " may not be " toy story , " bu...  0  1
 " a man is not a man without eight taels of go...  0  1
 " a private matter " is based on the true stor...  0  1
 " alcohol and drugs = bad . \nnot alcohol and ...  1  0
 " america's sweethearts " has an intriguing pr...  1  0
 " american pie " alums jason biggs and mena su...  1  0
 " an intimate , character-driven drama about a...  1  0
 " battlefield earth " is the best comedy of th...  1  0
 " be gentle , " urges natasha henstridge to ma...  1  0
 " being john malkovich " is the type of film w...  0  1
 " book " should have remained in shadows \nboo...  1  0
 " come on , silent bob ! \nwe're going to holl...  1  0
 " crazy/beautiful " suffers from the damned-if...  0  1
 " dangerous beauty " is a really nothing more ...  0  1
 " deep rising " gives you that sinking feeling...  1  0
 " desperate measures " is a generic title for ...  1  0
 " desperate measures " was something i was exc...  0  1
 " easely one of the worst films of the year . ...  1  0
 " first rule of fight club is , don't talk abo...  1  0
 " flubber " is the second best example of how ...  1  0
 " footloose " has only one goal in mind : to r...  0  1
 " from the director of " the nightmare before ...  1  0
 " gattaca " represents a solid breakthrough in...  0  1
 " good will hunting " is two movies in one : a...  0  1
 " goodbye , lover " sat on the shelf for almos...  1  0
 " gordy " is not a movie , it is a 90-minute-l...  1  0
 " have you ever heard the one about a movie so...  1  0
...                                                .. ..
yeah , yeah , the advertisements didn't even tr...  1  0
years ago , robin williams made _jumanji_ , a b...  1  0
yet another brainless teen flick , this one is ...  1  0
you always have to be careful with the first of...  1  0
you don't have to know poker to like " rounders...  0  1
you don't look at a ren ? magritte painting and...  1  0
you don't need to have seen the original " spec...  1  0
you don't want to be like mike . \nmike has bee...  0  1
you know , i never really wondered what the tar...  1  0
you know something , christmas is not about pre...  1  0
you know that a movie has issues when most ever...  1  0
you know the plot : a dimwit with a shady past ...  1  0
you know you're in for a truly different cinema...  0  1
you may not have heard of lucas , or if you hav...  0  1
you should have heard the old guys in the crowd...  1  0
you think that these people only exist in the m...  1  0
you would think that this film's dismal failure...  1  0
you'd think it would be easy to spoof star trek...  0  1
you'd think that after awhile hollywood would s...  1  0
you've got mail is a timely romance for this im...  0  1
you've got mail works alot better than it deser...  0  1
you've got to love disney . \nno matter what th...  0  1
you've got to think twice before you go see a m...  0  1
you've heard all the hype . \nyou've seen all t...  0  1
you've probably heard the one about the priest ...  0  1
you've seen this moment before , recently : a p...  0  1
young einstein is embarrassingly lame , but tha...  1  0
your first clue that something isn't gonna be q...  1  0
your friends and neighbors is a rather bizarre ...  0  1
zero effect gets its title from the main charac...  0  1

[2000 rows x 2 columns]

In [98]:
df.hist()

ValueError: num must be 1 <= num <= 0, not 1

In [ ]:
class_train.hist()